Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 файлам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
way = "/content/drive/MyDrive/Colab_Notebooks/MEVD/"

MessageError: ignored

In [ ]:
import pandas as pd
import numpy as np

# Задание 1 (3 балла)

1. Возьмите 10 000 случайных строк из таблицы sales_train и найдите в них долю товаров из категории 37. 
2. Найдите самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center shopping center за 2014 год.
3. Есть ли товар, который продается успешнее всех остальных (по ежемесячному количеству) во всех магазинах?

In [ ]:
st_data = pd.read_csv(way + "sales_train.csv", parse_dates=['date'], dayfirst=True, infer_datetime_format=True)
itemcat_data = pd.read_csv(way + "items.csv")
st_data["date"] = pd.to_datetime(st_data["date"])
st_data = st_data.assign(profit = st_data.item_cnt_day * st_data.item_price)
st_data.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,profit
0,2013-01-02,0,59,22154,999.00,1.0,999.00
1,2013-01-03,0,25,2552,899.00,1.0,899.00
2,2013-01-05,0,25,2552,899.00,-1.0,-899.00
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00
5,2013-01-10,0,25,2564,349.00,1.0,349.00
6,2013-01-02,0,25,2565,549.00,1.0,549.00
7,2013-01-04,0,25,2572,239.00,1.0,239.00
8,2013-01-11,0,25,2572,299.00,1.0,299.00
9,2013-01-03,0,25,2573,299.00,3.0,897.00


In [ ]:
#first
st_data_with_cat = st_data.merge(itemcat_data, on='item_id', how='left')
k = 10000
st_rand_data = st_data_with_cat.sample(k)
numberof_37 = st_rand_data.query("category_id == 37").shape[0]
ans_1 = numberof_37 / k
ans_1

0.0595

In [ ]:
#second
shops_data = pd.read_csv(way + "shops.csv")
shop_to_find = "St. Petersburg Nevsky Center shopping center"
shop_to_find_id = shops_data.query("shop_name.str.contains(@shop_to_find)", engine='python').iloc[0]["shop_id"]

st_data_profit_byday = st_data.query("shop_id == @shop_to_find_id & date >= '2014-01-01' & date < '2015-01-01'").groupby("date").agg({"profit" : "sum"})
st_data_profit_byday.loc[st_data_profit_byday['profit'].idxmax()]

profit    943833.0
Name: 2014-11-18 00:00:00, dtype: float64

In [ ]:
#third
grouped_st_data = st_data.groupby(['shop_id', 'item_id', pd.Grouper(key = "date", freq='1M')]) \
  .agg({"item_cnt_day": "sum"}).reset_index().set_index("shop_id")

grouped_st_data = grouped_st_data.groupby("shop_id", group_keys=False)

ans_items = grouped_st_data.apply(lambda x: x.sort_values(by = ["item_cnt_day"], ascending=False).head(1))
ans_items

,item_id,date,item_cnt_day
shop_id,,,
0,14752,2013-02-28,67.0
1,13354,2013-02-28,54.0
2,3731,2015-04-30,138.0
3,20949,2014-03-31,178.0
4,20949,2014-12-31,227.0
5,20949,2014-12-31,155.0
6,2814,2014-04-30,309.0
7,20949,2013-12-31,317.0
8,6738,2013-03-31,17.0


In [ ]:
ans_items_grouped = ans_items.groupby("item_id").agg({"item_cnt_day" : "count"}) \
  .rename({"item_cnt_day" : "number_of_shops"}, axis = 1).reset_index()
needed_item = ans_items_grouped.loc[ans_items_grouped["number_of_shops"].idxmax()].iloc[0]
items_data = pd.read_csv(way + "items.csv")
items_data.loc[needed_item].iloc[0]

'Corporate package T-shirt 1C Interest white (34 * 42) 45 microns'

# Задание 2 (2 балла)

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
st_data = pd.read_csv(way + "sales_train.csv", parse_dates=['date'], dayfirst=True, infer_datetime_format=True)
itemcat_data = pd.read_csv(way + "items.csv")
st_data["date"] = pd.to_datetime(st_data["date"])
st_data = st_data.assign(profit = st_data.item_cnt_day * st_data.item_price)
st_data.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,profit
0,2013-01-02,0,59,22154,999.00,1.0,999.00
1,2013-01-03,0,25,2552,899.00,1.0,899.00
2,2013-01-05,0,25,2552,899.00,-1.0,-899.00
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00
5,2013-01-10,0,25,2564,349.00,1.0,349.00
6,2013-01-02,0,25,2565,549.00,1.0,549.00
7,2013-01-04,0,25,2572,239.00,1.0,239.00
8,2013-01-11,0,25,2572,299.00,1.0,299.00
9,2013-01-03,0,25,2573,299.00,3.0,897.00


In [ ]:
#first
item_sales = st_data.groupby("item_id").agg({"item_cnt_day" : "sum"}).rename({"item_cnt_day" : "number_of_sales"}, axis = 1)
top_buy = item_sales.sort_values(["number_of_sales"]).tail(10)
itemcat_data[itemcat_data.item_id.isin(top_buy.index)]['item_name']

1855                   Battlefield 4 [PC, Russian version]
2808               Diablo III [PC, Jewel, Russian version]
3731            Grand Theft Auto V [PC, Russian subtitles]
3732           Grand Theft Auto V [PS3, Russian subtitles]
3734      Grand Theft Auto V [Xbox 360, Russian subtitles]
5822     Playstation Store replenishment of wallet: Pay...
6675     Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...
16787                    Some of us [PS3, Russian version]
17717                    Acceptance of funds for 1C-Online
20949    Corporate package T-shirt 1C Interest white (3...
Name: item_name, dtype: object

In [ ]:
#second
top_items_sales = st_data[st_data["item_id"].isin(top_buy.index)]
grouped_top_sales = top_items_sales.groupby(["item_id", top_items_sales.date.dt.year]).agg({"item_cnt_day" : "sum"})
number_of_years = len(st_data["date"].dt.year.unique())
item_correct = grouped_top_sales.groupby('item_id').apply(lambda x: len(x['item_cnt_day']) == number_of_years)
itemcat_data[itemcat_data.item_id.isin(item_correct.index) & item_correct == True]['item_name']

1855                   Battlefield 4 [PC, Russian version]
2808               Diablo III [PC, Jewel, Russian version]
3732           Grand Theft Auto V [PS3, Russian subtitles]
3734      Grand Theft Auto V [Xbox 360, Russian subtitles]
5822     Playstation Store replenishment of wallet: Pay...
6675     Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...
16787                    Some of us [PS3, Russian version]
17717                    Acceptance of funds for 1C-Online
20949    Corporate package T-shirt 1C Interest white (3...
Name: item_name, dtype: object

# Задание 3 (3 балла)

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

*Если таких категорий несколько, выведите все.*

In [ ]:
itemcat_data = pd.read_csv(way + "items.csv")
st_data = pd.read_csv(way + "sales_train.csv", parse_dates=['date'], dayfirst=True, infer_datetime_format=True)
category_data = pd.read_csv(way + "item_categories.csv")
itemcat_data

,item_name,item_id,category_id
0,!! IN THE POWER OF HAPPINESS (PLAST) D,0,40
1,! ABBYY FineReader 12 Professional Edition Ful...,1,76
2,*** IN THE GLORY OF THE GLORY (UNV) D,2,40
3,*** BLUE WAVE (Univ) D,3,40
4,*** BOX (GLASS) D,4,40
...,...,...,...
22165,"Nuclear Titbit 2 [PC, Digital Version]",22165,31
22166,Query language 1C: Enterprise [Digital version],22166,54
22167,The query language is 1C: Enterprise 8 (+ CD)....,22167,49
22168,Egg for Little Inu,22168,62


In [ ]:
category_data = category_data.rename({"item_category_id" : "category_id"}, axis = 1)
category_data

,item_category_name,category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4
...,...,...
79,Service,79
80,Service - Tickets,80
81,Blank media (spire),81
82,Blank media (piece),82


In [ ]:
item_sales = st_data.groupby("item_id").agg({"item_cnt_day" : "sum"}).rename({"item_cnt_day" : "number_of_sales"}, axis = 1)
category_sales = pd.concat([itemcat_data["item_id"], itemcat_data["category_id"], item_sales["number_of_sales"]], axis=1).set_index("item_id")
category_sales = category_sales.groupby("category_id").agg({"number_of_sales" : "sum"})

sorted_categories = category_sales.sort_values(["number_of_sales"])
sorted_categories

,number_of_sales
category_id,
51,1.0
10,1.0
1,2.0
53,3.0
52,3.0
...,...
37,203284.0
19,254887.0
55,348591.0


In [ ]:
min_category_value = sorted_categories.head(1).iloc[0].number_of_sales
max_category_value = sorted_categories.tail(1).iloc[0].number_of_sales
min_idxs = sorted_categories[sorted_categories['number_of_sales'] == min_category_value].index.tolist()
display(min_idxs)
max_idxs = sorted_categories[sorted_categories['number_of_sales'] == max_category_value].index.tolist()
display(max_idxs)
display(category_data[category_data.index.isin(min_idxs)])
display(category_data[category_data.index.isin(max_idxs)])

[51, 10]

[40]

,item_category_name,category_id
10,Game consoles - PS2,10
51,Books - Cognitive literature,51


,item_category_name,category_id
40,Cinema - DVD,40


# Задание 4 (3 балла)

1. Для категорий из задания 3 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами. Оставьте в таблице только те товары, среднее количество продаж которых строго больше 100.
2. Найдите месяц с максимальными продажами для каждого товара из пункта 1.


In [ ]:
st_data = pd.read_csv(way + "sales_train.csv", parse_dates=['date'], dayfirst=True, infer_datetime_format=True)
st_data["date"] = pd.to_datetime(st_data["date"])
st_data = st_data.assign(profit = st_data.item_cnt_day * st_data.item_price)
grouped_shops = st_data.groupby("shop_id").agg({"profit" : "sum"})
shops_sort = grouped_shops.sort_values(["profit"]).tail(5)
shops_sort

,profit
shop_id,
54,1.419397e+08
42,1.519170e+08
28,1.597463e+08
25,2.164806e+08
31,2.352170e+08


In [ ]:
#first
itemcat_data = pd.read_csv(way + "items.csv")
all_idxs = max_idxs + min_idxs
chosen_category_items = itemcat_data[itemcat_data.category_id.isin(all_idxs)]
chosen_category_items

,item_name,item_id,category_id
0,!! IN THE POWER OF HAPPINESS (PLAST) D,0,40
2,*** IN THE GLORY OF THE GLORY (UNV) D,2,40
3,*** BLUE WAVE (Univ) D,3,40
4,*** BOX (GLASS) D,4,40
5,*** NEW AMERICAN GRAPHICS (UNI) D,5,40
...,...,...,...
22158,YAMA (rem),22158,40
22159,YANKOVSKY,22159,40
22160,FAIR OF FAIRY (Region),22160,40
22162,THE FATHER,22162,40


In [ ]:
#matching_mincat_items = st_data.query("shop_id in @shops_sort.index & item_id in @min_category_items.index")
#table is empty
matching_cat_items = st_data.query("shop_id in @shops_sort.index & item_id in @chosen_category_items.index")
mean_of_cat_items = matching_cat_items.groupby("item_id").agg({"profit" : "mean"}).query("profit > 100")
mean_of_cat_items

,profit
item_id,
30,656.160870
32,321.183770
34,149.000000
35,568.412698
37,149.000000
...,...
22157,123.500000
22159,456.000000
22160,140.500000


In [ ]:
#second
month_sales = st_data.query("item_id in @mean_of_cat_items.index").groupby(["item_id", pd.Grouper(key = "date", freq='1M')]) \
  .agg({"profit" : "sum"}).reset_index()

max_profit_months = month_sales.groupby("item_id")["profit"].idxmax()
month_sales.loc[max_profit_months].set_index("item_id")

,date,profit
item_id,,
30,2013-02-28,332409.35
32,2013-01-31,101515.86
34,2013-06-30,2162.00
35,2013-02-28,43681.00
37,2013-04-30,3472.20
...,...,...
22157,2013-02-28,149.00
22159,2015-05-31,4788.00
22160,2013-01-31,1477.00


# Задание 5 (3 балла)

1. Для категорий из задания 3 найдите среднюю и медианную цену по категории в пределах каждого месяца. Выведите топ-10 товаров по медиане из каждой категории.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
#first
#max_category_items
matching_cat_items = st_data.query("item_id in @chosen_category_items.index")
matching_cat_items.groupby(pd.Grouper(key = "date", freq = "1M"))["item_price"].agg(["median", "mean"])

,median,mean
date,,
2013-01-31,199.00,244.485942
2013-02-28,203.00,252.174936
2013-03-31,231.00,260.645016
2013-04-30,199.00,254.925882
2013-05-31,199.00,247.627583
2013-06-30,199.00,245.860855
2013-07-31,149.00,236.812052
2013-08-31,149.00,234.470179
2013-09-30,198.00,241.818554


In [ ]:
median_cat_items = matching_cat_items.groupby("item_id").agg({"item_price" : "median"}).rename({"item_price" : "median_price"}, axis = 1)
median_cat_items.sort_values(["median_price"]).tail(10)

,median_price
item_id,
9640,849.00
9659,849.00
9698,849.00
14709,898.85
9580,899.00
9843,899.00
13993,999.00
19590,1299.00
20277,1399.00


In [ ]:
#second
mean_month_prices = st_data.groupby([pd.Grouper(key = "date", freq = "1Y"), pd.Grouper(key = "date", freq = "1M")]) \
  .agg({"item_price" : "mean"}).rename({"item_price" : "mean_price"}, axis = 1)
mean_month_prices.index.names = ['year', 'month']
mean_month_prices[(mean_month_prices["mean_price"] < mean_month_prices["mean_price"].shift(1)) & (mean_month_prices["mean_price"] < mean_month_prices["mean_price"].shift(-1))]
#mean_month_prices.query("(@mean_month_prices.mean_price < @mean_month_prices.mean_price.iloc[-1]) & (@mean_month_prices.mean_price < @mean_month_prices.mean_price.iloc[1])")

mean_price
year       month                  
2013-12-31 2013-05-31   624.179993
           2013-07-31   618.578247
2014-12-31 2014-01-31   862.869128
           2014-04-30   852.160167
           2014-07-31   847.046420
2015-12-31 2015-02-28  1010.465368
           2015-08-31   951.374139

# Доп задания

In [ ]:
#14
st_data = pd.read_csv(way + "sales_train.csv", parse_dates=['date'], dayfirst=True, infer_datetime_format=True)
itemcat_data = pd.read_csv(way + "items.csv")
st_data["date"] = pd.to_datetime(st_data["date"])

st_data_with_cat = st_data.merge(itemcat_data, on='item_id', how='left')
k = 10000
st_rand_data = st_data_with_cat.sample(k)
numberof_2 = st_rand_data.query("category_id == 2").shape[0]
ans_2 = numberof_2 / k
ans_2


0.0059

In [ ]:
#19
st_data_with_cat_sping = st_data_with_cat[(st_data_with_cat['date'].dt.month >= 3) & (st_data_with_cat['date'].dt.month <= 5)]
category_sales = st_data_with_cat_sping.groupby('category_id').agg({"item_cnt_day" : "sum"}).rename({'item_cnt_day' : 'sales'}, axis = 1)
category_sales = category_sales.sort_values(['sales'])
worst_categories_id = category_sales.head(4)
worst_categories_id

,sales
category_id,
0,1.0
48,1.0
27,1.0
10,1.0


In [ ]:
category_data = pd.read_csv(way + "item_categories.csv")
category_data = category_data.rename({"item_category_id" : "category_id"}, axis = 1)
category_data[category_data.index.isin(worst_categories_id.index)]

,item_category_name,category_id
0,PC - Headsets / Headphones,0
10,Game consoles - PS2,10
27,MAC Games - Number,27
48,Books - Computer literature,48


In [ ]:
worst_categories = st_data_with_cat_sping[st_data_with_cat_sping.category_id.isin(worst_categories_id.index)]
worst_categories

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,profit,item_name,category_id
361268,2013-04-17,3,26,6669,1998.0,1.0,1998.0,Sony PlayStation 2 (SCPH-77008),10
439377,2013-04-02,3,38,16256,22.4,1.0,22.4,Headphones RITMIX RH-120,0
482138,2013-05-30,4,35,6907,249.0,1.0,249.0,TeachPro Mathematics Wizard (BOOK + CD),48
2645135,2015-05-03,28,55,3872,449.0,1.0,449.0,"Hitman Absolution. Elite Edition [MAC, Digital...",27


In [ ]:
worst_categories.groupby(worst_categories['date'].dt.strftime('%B')).agg({'item_price' : 'mean'})

,item_price
date,
April,1010.2
May,349.0
